# Compile Beats Dataset

In [17]:
#library imports: 
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot
import libpysal as lp
import shapely

In [4]:
#Run helper script to get helper functions
%run geoprocessing_helpers.ipynb

../raw_data/Boundaries - Police Beats (current).geojson EPSG:3435
../raw_data/Boundaries - Census Blocks - 2010.geojson EPSG:3435


# Importation and Data Setup:

In [19]:
#Read Data
beats = read_spatial("../raw_data/Boundaries - Police Beats (current).geojson")
l = read_spatial("../raw_data/CTA_RailStations")
bus = read_spatial("../raw_data/CTA_BusStops")
metra = read_spatial("../raw_data/Metra_Stations")
streets = read_spatial("../raw_data/Street Center Lines.geojson")
bus_data = read_spatial("../intermediate_data/bus_geog")

#Separate business data
rest = bus_data[bus_data["category"]=="restaurant"]
bar = bus_data[bus_data["category"]=="bar"]
business = bus_data[bus_data["category"]=="business"]
daycare = bus_data[bus_data["category"]=="daycare"]
entertainment = bus_data[bus_data["category"]=="entertainment"]

../raw_data/Boundaries - Police Beats (current).geojson EPSG:3435
../raw_data/CTA_RailStations epsg:3435
../raw_data/CTA_BusStops EPSG:3435
../raw_data/Metra_Stations epsg:3435
../raw_data/Street Center Lines.geojson EPSG:3435
../intermediate_data/bus_geog EPSG:3435


# Aggregate Point to Beats Data

In [20]:
beats = join_to_beats(beats, "beat_num", l, "count_l_stops")
beats = join_to_beats(beats, "beat_num", bus, "count_bus_stops")
beats = join_to_beats(beats, "beat_num", metra, "count_metra_stops")
beats = join_to_beats(beats, "beat_num", rest, "count_restaurants")
beats = join_to_beats(beats, "beat_num", bar, "count_bars")
beats = join_to_beats(beats, "beat_num", daycare, "count_daycares")
beats = join_to_beats(beats, "beat_num", entertainment, "count_entertainment")
beats = join_to_beats(beats, "beat_num", business, "count_businesses")

# Handle Street Data

In [21]:
#Filter Out non-pedestrian Streets (ie highways):
#Syntax help: https://www.kite.com/python/answers/how-to-filter-a-pandas-dataframe-with-a-list-by-%60in%60-or-%60not-in%60-in-python
streets_do_not_want = ["ER", "EXPY", "HWY", "PKWY", "ORD", "XR", "TOLL"]
street_filter = ~streets.street_typ.isin(streets_do_not_want)
streets = streets[street_filter]

In [22]:
#Spatial operation to get all strings into exactly one polygon
#Help on overlay code here::https://stackoverflow.com/questions/60794994/geopandas-split-lines-at-intersection-with-polygons-retain-polygon-id-in-new
#Help with renaming after groupby: https://stackoverflow.com/questions/44416287/renaming-columns-after-group-by-and-sum-in-pandas-dataframe

#First: separate line segments so that they fall into exactly one police beat
#Second: same spatial join as above
#Third: Compute distance (in feet) of each segment
#Fourth/Fifth: Keep only the beat_num and distance columns
#Sixth:group by beat and sum all distances in the beat
#seventh:Add the column with the summarized distances back to the beat dataset


streets_separated = gpd.overlay(streets, beats, how="union", keep_geom_type=False).explode().reset_index(drop=True)
streets_separated = gpd.sjoin(streets_separated, 
                             beats[["beat_num", "geometry"]], 
                             how="inner", 
                             op='intersects')
streets_separated["distance"]= streets_separated["geometry"].length
streets_separated = streets_separated[["beat_num_left", "distance"]]
streets_separated.rename(columns={"beat_num_left":"beat_num"}, inplace=True)
streets_separated = streets_separated.groupby("beat_num").sum("distance").reset_index().rename(columns={'distance':'road_distance_ft'})
beats = beats.merge(streets_separated, how ="left", on="beat_num")

# Crosswalk to Beats From Blocks: 

In [29]:
#Create Crosswalk From Block to Beat:
#Read Block spatial data and population by block
#Then join the two
blocks = read_spatial("../raw_data/Boundaries - Census Blocks - 2010.geojson")
pop_10 = pd.read_csv("../raw_data/Population_by_2010_Census_Block.csv", 
                     dtype={"CENSUS BLOCK FULL":"object", "CENSUS BLOCK":"object"})
blocks = blocks.merge(pop_10, left_on="geoid10", right_on="CENSUS BLOCK FULL", how="left")

../raw_data/Boundaries - Census Blocks - 2010.geojson EPSG:3435


In [32]:
beats = convert_block_to_beat(blocks, "TOTAL POPULATION")

# Calculate Distance to Nearest:

In [33]:
beats["centroid"]=beats.centroid
beats = dist_to_nearest(beats, police_station, "centroid", "dist_to_police")
beats = dist_to_nearest(beats, hospital, "centroid", "dist_to_hospital")

# Spatial Lag

In [ ]:
#NEED TO DO THIS once we get crime code

# Write Data

In [34]:
beats.columns
#ADD MORE HERE WHEN WE DO SPATIAL LAG!
beats_to_write = beats[['beat', 'beat_num', 'district', 'sector', 'count_l_stops',
                'count_bus_stops', 'count_metra_stops', 'count_restaurants',
       'count_bars', 'count_daycares', 'count_entertainment',
       'count_businesses', 'road_distance_ft', 'TOTAL POPULATION',
       'dist_to_police', 'dist_to_hospital']]

beats_to_write.to_csv("../intermediate_data/beats.csv")

